In [1]:
from multi_stock_trading_env import MultiStockTradingEnv

from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3 import PPO

from gym_anytrading.envs import StocksEnv

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

matplotlib.use('Qt5Agg')

In [2]:
checkpoint_callback = CheckpointCallback(save_freq=1e4, save_path='./model_checkpoints/')

In [3]:
df = pd.read_csv("hackathon_sample_v2.csv")
dfList = []
stockNames = df['stock_ticker'].unique().tolist()

for ticker in stockNames:
    dfList.append(df[df['stock_ticker'] == ticker])

df = pd.concat(dfList)

In [4]:
df

,date,ret_eom,permno,shrcd,exchcd,mspread,year,month,size_port,rf,...,betadown_252d,bidaskhl_21d,corr_1260d,betabab_1260d,rmax5_rvol_21d,age,qmj,qmj_prof,qmj_growth,qmj_safety
0,20000131,20000131,10078,11,3,0.017178,2000,1,B,0.0041,...,2.219037,0.012635,0.504688,1.581540,1.373224,180,1.700939,1.711756,1.476410,1.087626
1242,20000229,20000229,10078,11,3,0.011570,2000,2,B,0.0043,...,1.930644,0.017178,0.516748,1.593141,1.618713,181,1.639150,1.671388,1.084789,1.105849
2471,20000331,20000331,10078,11,3,0.016430,2000,3,B,0.0047,...,1.602455,0.011570,0.516163,1.555353,1.384841,182,1.644516,1.674167,1.089909,1.113471
3790,20000428,20000430,10078,11,3,0.022907,2000,4,B,0.0046,...,1.664166,0.016430,0.520867,1.509584,1.354060,183,1.644663,1.673487,1.083610,1.106420
5064,20000531,20000531,10078,11,3,0.019930,2000,5,B,0.0050,...,1.246692,0.022907,0.534530,1.380081,2.171063,184,1.674932,1.699119,1.130745,1.197280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272737,20231229,20231231,21747,11,3,0.011811,2023,12,B,0.0043,...,1.810903,0.004645,NaN,NaN,0.882738,59,NaN,-1.147374,NaN,1.089865
271767,20231130,20231130,23439,11,3,0.011642,2023,11,B,0.0044,...,0.847020,0.014562,NaN,NaN,1.264852,58,NaN,0.098592,NaN,1.552123
272771,20231229,20231231,23439,11,3,0.009993,2023,12,B,0.0043,...,0.962572,0.011642,NaN,NaN,0.878537,59,NaN,0.089709,NaN,1.557396
272332,20231130,20231130,91733,11,3,0.003967,2023,11,B,0.0044,...,0.886458,0.008851,0.458541,1.261573,1.146990,262,-0.471809,0.275254,-0.586187,0.006212


: 

In [ ]:
df = df.fillna(0)

In [5]:
df['prc'].plot()

In [ ]:
df[df['stock_ticker'] == 'AAPL']['prc'].plot()

In [ ]:
len(df['stock_ticker'].unique().tolist())

In [9]:
def personal_process_data(df, window_size, stockTickers, frame_bound):
    start = frame_bound[0] - window_size
    end = frame_bound[1]

    prices = df.loc[:, 'prc'].to_numpy()[start:end]
    signal_features = df.loc[:, [
            'age', 'aliq_at', 'aliq_mat', 'ami_126d', 'at_be', 'at_gr1', 'at_me', 'at_turnover', 'be_gr1a', 'be_me', 'beta_60m', 'beta_dimson_21d', 'betabab_1260d', 
            'betadown_252d', 'bev_mev', 'bidaskhl_21d', 'capex_abn', 'capx_gr1', 'capx_gr2', 'capx_gr3', 'cash_at', 'chcsho_12m', 'coa_gr1a', 'col_gr1a', 'cop_at', 
            'cop_atl1', 'corr_1260d', 'coskew_21d', 'cowc_gr1a', 'dbnetis_at', 'debt_gr3', 'debt_me', 'dgp_dsale', 'div12m_me', 'dolvol_126d', 'dolvol_var_126d', 
            'dsale_dinv', 'dsale_drec', 'dsale_dsga', 'earnings_variability', 'ebit_bev', 'ebit_sale', 'ebitda_mev', 'emp_gr1', 'eq_dur', 'eqnetis_at', 'eqnpo_12m', 
            'eqnpo_me', 'eqpo_me', 'f_score', 'fcf_me', 'fnl_gr1a', 'gp_at', 'gp_atl1', 'intrinsic_value', 'inv_gr1', 'inv_gr1a', 'iskew_capm_21d', 'iskew_ff3_21d', 
            'iskew_hxz4_21d', 'ivol_capm_21d', 'ivol_capm_252d', 'ivol_ff3_21d', 'ivol_hxz4_21d', 'kz_index', 'lnoa_gr1a', 'lti_gr1a', 'market_equity', 'mispricing_mgmt', 
            'mispricing_perf', 'ncoa_gr1a', 'ncol_gr1a', 'netdebt_me', 'netis_at', 'nfna_gr1a', 'ni_ar1', 'ni_be', 'ni_inc8q', 'ni_ivol', 'ni_me', 'niq_at', 'niq_at_chg1', 
            'niq_be', 'niq_be_chg1', 'niq_su', 'nncoa_gr1a', 'noa_at', 'noa_gr1a', 'o_score', 'oaccruals_at', 'oaccruals_ni', 'ocf_at', 'ocf_at_chg1', 'ocf_me', 
            'ocfq_saleq_std', 'op_at', 'op_atl1', 'ope_be', 'ope_bel1', 'opex_at', 'pi_nix', 'ppeinv_gr1a', 'prc', 'prc_highprc_252d', 'qmj', 'qmj_growth', 
            'qmj_prof', 'qmj_safety', 'rd_me', 'rd_sale', 'rd5_at', 'resff3_12_1', 'resff3_6_1', 'ret_1_0', 'ret_12_1', 'ret_12_7', 'ret_3_1', 'ret_6_1', 
            'ret_60_12', 'ret_9_1', 'rmax1_21d', 'rmax5_21d', 'rmax5_rvol_21d', 'rskew_21d', 'rvol_21d', 'sale_bev', 'sale_emp_gr1', 'sale_gr1', 'sale_gr3', 
            'sale_me', 'saleq_gr1', 'saleq_su', 'seas_1_1an', 'seas_1_1na', 'seas_2_5an', 'seas_2_5na', 'sti_gr1a', 'taccruals_at', 'taccruals_ni', 
            'tangibility', 'tax_gr1a', 'turnover_126d', 'turnover_var_126d', 'z_score', 'zero_trades_126d', 'zero_trades_21d', 'zero_trades_252d'
        ]].to_numpy()[start:end]

    return prices, signal_features

class PersonalStockEnv(StocksEnv):
    def __init__(self, prices, signal_features, **kwargs):
        self.prices = prices
        self.signal_features = signal_features
        return super(PersonalStockEnv, self).__init__(**kwargs)

    def _process_data(self):
        return self.prices, self.signal_features


In [10]:
prices, signal_features = personal_process_data(df=df, window_size=30, stockTickers=stockNames, frame_bound=(30, len(df)))
env = PersonalStockEnv(prices, signal_features, df=df, window_size=30, frame_bound=(30, len(df)))

In [ ]:
model = PPO("MlpPolicy", env, tensorboard_log='./logs/saved_models/', verbose=1) 
model.learn(total_timesteps=10_000)
model.save("trading_bot")

In [ ]:
prices, signal_features = personal_process_data(df=df[df['stock_ticker'] == 'AAPL'], window_size=30, stockTickers=stockNames, frame_bound=(30, len(df[df['stock_ticker'] == 'AAPL'])))
env = PersonalStockEnv(prices, signal_features, df=df[df['stock_ticker'] == 'AAPL'], window_size=30, frame_bound=(30, len(df[df['stock_ticker'] == 'AAPL'])))
model = PPO.load("trading_bot")

obs, _ = env.reset()
while True: 
    obs = obs[np.newaxis, ...]
    action, _ = model.predict(obs)
    obs, rewards, terminated, truncated, info = env.step(action)
    done = terminated or truncated
    if done:
        print("info", info)
        break

plt.figure(figsize=(15,6))
plt.cla()
env.render_all()
plt.show()